In [1]:
---- MODULE mm_recovery ----

EXTENDS Integers, Sequences, FiniteSets, TLC
VARIABLES state, logs

CONSTANTS nodes


n_nodes == Cardinality(nodes)


\**************************************************************************************
\* Helpers
\**************************************************************************************


quorum(mask) == Cardinality({i \in DOMAIN mask : mask[i] = 1}) >= (n_nodes \div 2 + 1)

max(set) == IF set = {} THEN 0 ELSE CHOOSE e1 \in set: \A e2 \in set: e1 >= e2

maxlen(seqs) == max({Len(seqs[n]) : n \in DOMAIN seqs})

maxlsn(log, origin) == max({log[j].olsn : j \in {i \in DOMAIN log : log[i].origin = origin }})

rep_state(log) == [n \in nodes |-> maxlsn(log,n)]

log_older_then(log, origin_vec) == SelectSeq(log, LAMBDA e: e.olsn > origin_vec[e.origin])
    
\*is_increasing(s) == IF Len(s) > 1
\*                    THEN {s[i] < s[i+1] : i \in 1..(Len(s)-1)} = {TRUE}
\*                    ELSE TRUE

new_status(old_status, vmask, emask) ==
    CASE ~ quorum(vmask) -> "disabled"
    [] old_status = "disabled" -> "recovery"
    \* recovery -> recovered done explicetly in do_recovery()
    [] (old_status = "recovered" /\ vmask = emask) -> "online"
    [] OTHER -> old_status


\**************************************************************************************
\* Initial
\**************************************************************************************


init == /\ state = [n \in nodes |-> [
                        next_lsn |-> 1,
                        status |-> "disabled",
                        view |-> [[_n \in nodes |-> 0] EXCEPT ![n] = 1],
                        enabled |-> [[_n \in nodes |-> 0] EXCEPT ![n] = 1]
                    ]]
        /\ logs =  [n \in nodes |-> << >>]

\**************************************************************************************
\* Actions
\**************************************************************************************


disconnect(n1, n2) ==
    /\ n1 /= n2
    /\ state[n1].view[n2] = 1
    
    /\ logs' = logs
    /\  LET
            view == [state[n1].view EXCEPT ![n2] = 0]
            emask == [state[n1].enabled EXCEPT ![n2] = 0]
            status == new_status(state[n1].status, view, emask)
        IN
        state' = [state EXCEPT  ![n1].view = view,
                                ![n1].enabled = emask,
                                ![n1].status = status]


connect(n1, n2) ==
    /\ n1 /= n2
    /\ state[n1].view[n2] = 0

    /\ logs' = logs
    /\  LET
            view == [state[n1].view EXCEPT ![n2] = 1]
            status == new_status(state[n1].status, view, state[n1].enabled)
        IN
        state' = [state EXCEPT  ![n1].view = view,
                                ![n1].status = status]

do_recovery(n1, n2) ==
    /\ n1 /= n2
    /\ state[n1].status = "recovery"
    /\ state[n1].view[n2] = 1
    /\ state[n2].view[n1] = 1

    /\  LET
            origin_vec == rep_state(logs[n1])
            new_entries == log_older_then(logs[n2], origin_vec)
            n2_emask == [state[n2].enabled EXCEPT ![n1] = 1]
            n2_status == new_status(state[n2].status, state[n2].view, n2_emask)
        IN
        /\ logs' = [logs EXCEPT ![n1] = logs[n1] \o new_entries]
        /\ state' = [state EXCEPT   ![n1].status = "recovered",
                                ![n2].enabled = n2_emask,
                                ![n2].status = n2_status]


do_recovered(n1, n2) ==
    /\ n1 /= n2
    /\ (state[n1].status = "recovered" \/ state[n1].status = "online")
    /\ state[n1].view[n2] = 1
    /\ state[n2].view[n1] = 1

    /\  LET
            our_last_lsn == maxlsn(logs[n1], n2)
            new_entries == SelectSeq(logs[n2], LAMBDA e: e.origin = n2 /\ e.olsn > our_last_lsn )
        IN
            logs' = [logs EXCEPT ![n1] = logs[n1] \o new_entries]
    /\  LET
            n2_emask == [state[n2].enabled EXCEPT ![n1] = 1]
            n2_status == new_status(state[n2].status, state[n2].view, n2_emask)
        IN
        state' = [state EXCEPT  ![n2].enabled = n2_emask,
                                ![n2].status = n2_status]


do_tx(node) ==
    /\ state[node].status = "online"
    /\ logs' = [n \in nodes |->
                    IF state[node].enabled[n] = 1
                    THEN Append(logs[n], [origin |-> node, olsn |-> state[node].next_lsn])
                    ELSE logs[n]]
    /\ state' = [state EXCEPT ![node].next_lsn = state[node].next_lsn + 1]


\**************************************************************************************
\* Final spec
\**************************************************************************************


next == \*/\  
            \/ \E n1,n2 \in nodes : connect(n1,n2)
            \/ \E n1,n2 \in nodes : disconnect(n1,n2)
            \/ \E n1,n2 \in nodes : do_recovery(n1,n2)
            \/ \E n1,n2 \in nodes : do_recovered(n1,n2)
            \/ \E n \in nodes : do_tx(n)

\*         /\ maxlen(logs) < 3

spec == init /\ [][next]_<<state, logs>>


\**************************************************************************************
\* Stuff to check
\**************************************************************************************

same_data == \A n1,n2 \in {n \in nodes : state[n].status = "online"} : logs[n1] = logs[n2]
                                
smtr == Permutations(nodes)

\* THEOREM spec => []( no_loss )

====

In [2]:
%tlc:mm_recovery
CONSTANT nodes = {node1,node2,node3}
\* SYMMETRY smtr
INVARIANT same_data
SPECIFICATION spec

TLC2 Version 2.14 of 10 July 2019 (rev: 0cae24f)
Running breadth-first search Model-Checking with fp 35 and seed -3174544884984667607 with 8 workers on 8 cores with 3641MB heap and 64MB offheap memory (Mac OS X 10.15.1 x86_64, Oracle Corporation 1.8.0_112 x86_64, MSBDiskFPSet, DiskStateQueue).
Parsing file /private/var/folders/hq/xxjfdhms2lv461cpv_6dc6th0000gn/T/tmpo2zjyja3/mm_recovery.tla
Parsing file /private/var/folders/hq/xxjfdhms2lv461cpv_6dc6th0000gn/T/Integers.tla
Parsing file /private/var/folders/hq/xxjfdhms2lv461cpv_6dc6th0000gn/T/Sequences.tla
Parsing file /private/var/folders/hq/xxjfdhms2lv461cpv_6dc6th0000gn/T/FiniteSets.tla
Parsing file /private/var/folders/hq/xxjfdhms2lv461cpv_6dc6th0000gn/T/TLC.tla
Parsing file /private/var/folders/hq/xxjfdhms2lv461cpv_6dc6th0000gn/T/Naturals.tla
Semantic processing of module Naturals
Semantic processing of module Integers
Semantic processing of module Sequences
Semantic processing of module FiniteSets
Semantic processing of module TLC
S